## Advanced RAG

In [91]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

384


In [15]:
documents[0].metadata

{'source': 'cookbook/sql_db_qa.mdx',
 'file_path': 'cookbook/sql_db_qa.mdx',
 'file_name': 'sql_db_qa.mdx',
 'file_type': '.mdx'}

In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings, persist_directory="./chroma")

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクターストアなど）に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。これにより、開発者は異なるプロバイダー間での切り替えが容易になります。'

## 6.3. 検索クエリの工夫
### HyDE（Hypothetical Document Embeddings）

In [17]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。

質問: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

In [18]:
hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、アプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築することができます。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイメント**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、さまざまなプロバイダーと統合し、標準インターフェースを実装しているため、異なるコンポーネントプロバイダーを簡単に切り替えることができます。また、LangGraphを使用することで、複雑なアプリケーションのオーケストレーションが可能になり、記憶やストリーミングなどの重要な機能をサポートします。LangSmithは、アプリケーションのトレースや評価を行うためのプラットフォームです。\n\n全体として、LangChainは開発者がLLMアプリケーションを効率的に構築、評価、デプロイできるように設計されています。'

In [19]:
hypothetical_chain.invoke("LangChainの概要を教えて")

'LangChainは、言語モデルを活用したアプリケーションの開発を容易にするためのフレームワークで、データの取得、処理、出力の各ステップを統合し、対話型エージェントや自動化ツールの構築をサポートします。'

## 複数の検索クエリの生成

In [66]:
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")


query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。

質問: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

In [67]:
multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、開発、運用、デプロイの各段階を簡素化することを目的としています。具体的には、以下のような特徴があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **運用**: LangSmithを使用してアプリケーションを監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、さまざまなモデルや関連コンポーネントの標準化されたインターフェースを提供し、開発者が異なるプロバイダー間で簡単に切り替えたり、コンポーネントを組み合わせたりできるようにします。また、アプリケーションの複雑さが増すにつれて、オーケストレーションや可視化、評価のニーズが高まることに対応しています。\n\n全体として、LangChainは、開発者がAIアプリケーションを効率的に構築、運用、デプロイできるようにするための強力なエコシステムを提供しています。'

In [68]:
query_generation_chain.invoke("LangChainの概要を教えて")

['LangChainとは何か、その基本的な機能と用途について',
 'LangChainの主要なコンポーネントとアーキテクチャの説明',
 'LangChainを使用した具体的なアプリケーション例とその利点']

In [72]:
query_results_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
    | retriever.map()
    # | (lambda docs_list: [doc.page_content for doc in docs_list[0]])
)

In [73]:
result = query_results_chain.invoke("LangChainの概要を教えて")

In [88]:
array1 = np.array([-18, -18, -18])
array2 = np.array([0, 0, 0])

# 連結
result = np.concatenate((array1, array2))


In [89]:
result

array([-18, -18, -18,   0,   0,   0])

In [94]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document

def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    question = inp['question']
    documents = inp['documents']

    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)
    return cohere_reranker.compress_documents(documents=documents, query=question)

rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    | prompt | model | StrOutputParser()
)

rerank_rag_chain.invoke('Langchainの概要を教えて')

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなどの関連技術に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。また、複数のオープンソースライブラリで構成されており、開発者は必要なコンポーネントを選択して使用することができます。'

In [95]:
from langchain_community.retrievers import TavilySearchAPIRetriever

# langsmithでトレースしやすいようにconfigをつける
langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [101]:
from enum import Enum

class Route(str, Enum):
    langchain_document = "langchain_document"
    web = 'web'

class RouteOutput(BaseModel):
    route: Route


route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。

質問: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
    
)

In [102]:
class Route(Enum):
    langchain_document = "langchain_document"
    web = 'web'

In [107]:
isinstance(Test.web, str)

False

In [108]:
isinstance(Route.web, str)

True

In [109]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unknown retriever: {retriever}")


route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

In [110]:
route_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクターストアなど）に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。また、複数のオープンソースライブラリで構成されており、特に`langchain-core`、統合パッケージ、`langchain`、`langchain-community`、`langgraph`などが含まれています。\n\nさらに、LangChainは、開発者がアプリケーションを構築する際に直面するさまざまな課題に対処するための標準化されたコンポーネントインターフェース、オーケストレーション機能、可視性と評価のサポートを提供しています。'

In [111]:
route_rag_chain.invoke("東京の今日の天気は？")

'東京の今日の天気は、10月27日（日）で最高気温24℃、最低気温17℃、降水確率は50%です。'

## ハイブリッド検索

In [127]:
def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 各ドキュメントのコンテンツ (文字列) とそのスコアの対応を保持する辞書を準備
    content_score_mapping = {}

    # 検索クエリごとにループ
    for docs in retriever_outputs:
        # 検索結果のドキュメントごとにループ
        for rank, doc in enumerate(docs):
            content = doc.page_content

            # 初めて登場したコンテンツの場合はスコアを0で初期化
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (順位 + k)) のスコアを加算
            content_score_mapping[content] += 1 / (rank + k)

    # スコアの大きい順にソート
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # noqa
    return [content for content, _ in ranked]

In [126]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config(
    {"run_name": "chroma_retriever"}
)

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

In [132]:
from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel({
        "chroma_documents": chroma_retriever,
        "bm25_documents": bm25_retriever,
    })
    # | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    # | reciprocal_rank_fusion
)

In [130]:
hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChainの概要を教えて")

AttributeError: 'dict' object has no attribute 'invoke'

In [133]:
hybrid_retriever.invoke("LangChainの概要を教えて")

{'chroma_documents': [Document(metadata={'file_name': 'introduction.mdx', 'file_path': 'docs/docs/introduction.mdx', 'file_type': '.mdx', 'source': 'docs/docs/introduction.mdx'}, page_content='---\nsidebar_position: 0\nsidebar_class_name: hidden\n---\n\n# Introduction\n\n**LangChain** is a framework for developing applications powered by large language models (LLMs).\n\nLangChain simplifies every stage of the LLM application lifecycle:\n- **Development**: Build your applications using LangChain\'s open-source [components](/docs/concepts) and [third-party integrations](/docs/integrations/providers/).\nUse [LangGraph](/docs/concepts/architecture/#langgraph) to build stateful agents with first-class streaming and human-in-the-loop support.\n- **Productionization**: Use [LangSmith](https://docs.smith.langchain.com/) to inspect, monitor and evaluate your applications, so that you can continuously optimize and deploy with confidence.\n- **Deployment**: Turn your LangGraph applications into p